# Bike Sales Analysis

*A Python-based exploratory analysis of sales performance, customer demographics, and product profitability.*

## Project Overview
This project analyzes **bike sales data** using Python to identify trends in revenue, profit, customer behavior, and product performance.  
The goal is to generate **business-focused insights** that support data-driven decision-making.

## Tools Used
- Python  
- pandas, NumPy  
- Matplotlib, Seaborn  
- Jupyter Notebook

# 1. Importing Tools & Loading the Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the dataset
bike_sales_df = pd.read_csv("Bike_Sales.csv")

# Inspecting the first 5 rows
bike_sales_df.head()

In [ ]:
# Reveal the shape of the dataframe
bike_sales_df.shape

The dataset has been successfully loaded and contains 113036 rows and 11 columns.

#  2. Data Overview & Understanding

In [ ]:
# Show the info about columns
bike_sales_df.info()

The dataset contains **transaction-level bike sales data**, including customer information, geographic details, and product attributes.

Each row represents a single sales transaction.

In [ ]:
# Show major aggregations
bike_sales_df.describe()

In [ ]:
# Check number of missing values
bike_sales_df.isnull().sum()

The dataset has no missing values.

In [ ]:
# Check number of duplicates
bike_sales_df.duplicated().sum()

The dataset contains approximately 1,000 duplicate rows.

# 3. Data Cleaning & Preparation

The `Date` column is converted to a datetime format to enable time-based analysis.  
Other columns are reviewed to ensure appropriate data types for analysis.

In [ ]:
# Transform the type of date column to datetime
bike_sales_df['Date'] = pd.to_datetime(bike_sales_df['Date'])

# To check
# bike_sales_df['Date'].astype

As confirmed in the data overview, the dataset contains no missing values.

Since there is no unique order identifier, these records may represent transactions made by different customers with identical details.  
Therefore, the duplicate rows were retained to avoid underestimating sales and revenue.

### Adding some useful columns

Additional features are created to support deeper analysis, including:
- Age groups for customers. 
- Seasons, Year_Month for each transaction.  
- The cost, revenue and profit of each transaction.

In [ ]:
# Creating the month column
bike_sales_df["Year_Month"] = bike_sales_df["Date"].dt.to_period("M").dt.to_timestamp()

# Inserting it in the right position
bike_sales_df.insert(1, "Year_Month", bike_sales_df.pop("Year_Month"))

In [ ]:
# Get the season from the date
def get_season(date):
    month = date.month
    day = date.day

    if (month==3 and day>=21) or (month==4 or month==5) or (month==6 and day<21):
        return 'Spring'
    elif (month==6 and day>=21) or (month==7 or month==8) or (month==9 and day<23):
        return 'Summer'
    elif (month==9 and day>=23) or (month==10 or month==11) or (month==12 and day<21):
        return 'Fall'
    else:
        return 'Winter'

# Creating the season column
bike_sales_df["Season"] = bike_sales_df["Date"].apply(get_season)

# Inserting it in the right position
bike_sales_df.insert(2, "Season", bike_sales_df.pop("Season"))


In [ ]:
# Get the age group from customer age
def get_age_group(age):
    if age<25:
        return "Youth (<25)"
    elif age<35:
        return "Young Adults (25-34)"
    elif age<65:
        return "Adults (35-64)"
    else:
        return "Seniors (64+)"

# Creating the age group column
bike_sales_df["Age Group"] = bike_sales_df["Customer_Age"].apply(get_age_group)

# Inserting it in the right position
bike_sales_df.insert(4, "Age Group", bike_sales_df.pop("Age Group"))

In [ ]:
# Creating the transaction cost column
bike_sales_df["Transaction_Cost"] = bike_sales_df["Unit_Cost"] * bike_sales_df["Order_Quantity"]

In [ ]:
# Creating the transaction revenue column
bike_sales_df["Transaction_Revenue"] = bike_sales_df["Unit_Price"] * bike_sales_df["Order_Quantity"]

In [ ]:
# Creating the transaction profit column
bike_sales_df["Transaction_Profit"] = bike_sales_df["Transaction_Revenue"] - bike_sales_df["Transaction_Cost"]
bike_sales_df.head()

# 4. Exploratory Data Analysis (EDA)

In this section, we are going to explore overall patterns and trends in the bike sales data.  

This exploratory analysis helps identify key areas of interest and guides the selection of business insights analyzed in the next section.

## Sales Growth Over Time
This analysis examines how total sales evolve over time to identify overall growth trends and potential seasonality in bike sales.

In [ ]:
# Aggregate monthly revenue
Sales_over_time = (
    bike_sales_df
    .groupby("Year_Month")["Transaction_Revenue"]
    .sum()
    .sort_index()
)

# Plot
plt.figure(figsize=(10,5))
plt.plot(Sales_over_time)
plt.xlabel("Year-Month")
plt.ylabel("Total Revenue ($)")
plt.title("Total Revenue Over Time")
plt.tight_layout()
plt.show()


Revenue has grown significantly over time, suggesting a rising demand for bike products across the analyzed period.

## Sales by Country
This analysis compares sales performance across different countries to understand geographic differences in demand.

In [ ]:
# Aggregate revenue by country
Sales_by_country = ( 
    bike_sales_df.groupby("Country")["Transaction_Revenue"]
    .sum()
    .sort_values(ascending=False)
)

# Plot
plt.figure(figsize=(10,5))
plt.bar(Sales_by_country.index, Sales_by_country.values, color='skyblue')
plt.xlabel("Country")
plt.ylabel("Total Revenue ($)")
plt.title("Total Revenue By Country")
plt.tight_layout()
plt.show()

USA and Australia generate the highest revenue, indicating strong bike demand in these markets.

## Sales by Customer Age Group
This analysis explores how sales vary across customer age groups to identify which segments contribute most to overall sales.

In [ ]:
# Aggregate revenue by age
Sales_by_age_group = bike_sales_df.groupby("Age Group")["Transaction_Revenue"].sum().sort_values(ascending=False)

# Plot
plt.figure(figsize=(10,5))
plt.bar(Sales_by_age_group.index, Sales_by_age_group.values, color='skyblue')
plt.xlabel("Age Group")
plt.ylabel("Total Revenue ($)")
plt.title("Total Revenue By Age Group")
plt.tight_layout()
plt.show()

Adults (35-64) generate the highest revenue, while Seniors (65+) contribute the least, indicating that the majority of bike sales come from the adult age segment.

## Sales by Product Category
This analysis reviews sales performance across product categories to understand which types of products drive the highest sales volume.

In [ ]:
# Aggregate revenue by product category
Sales_by_category = (
    bike_sales_df.groupby("Product_Category")["Transaction_Revenue"]
    .sum()
    .sort_values(ascending=False)
)

# Plot
plt.figure(figsize=(10,5))
plt.bar(Sales_by_category.index, Sales_by_category.values, color='skyblue')
plt.xlabel("Product Category")
plt.ylabel("Total Revenue ($)")
plt.title("Total Revenue By Category")
plt.tight_layout()
plt.show()

Bike products generate the highest total revenue, likely influenced by their higher unit prices compared to other categories.

# 5. Business Insights & Visual Analysis

In this section, to identify the key factors that drive **profitability** in bike sales.  
The analysis focuses on understanding which segments, products, and time periods generate the highest profit.

## Main Business Question
**What factors generate the most profit in bike sales?**

### Time and Place

1. Profit by season

In [ ]:
Profit_by_Season = ( 
    bike_sales_df.groupby("Season")["Transaction_Profit"]
    .sum()
    .sort_values(ascending=False)
)

# Plot
plt.figure(figsize=(10, 5))
plt.bar(Profit_by_Season.index, Profit_by_Season.values, color='skyblue')
plt.xlabel("Season")
plt.ylabel("Total Profit ($)")
plt.title("Total Profit By Season")
plt.tight_layout()
plt.show()

Spring generates the highest total profit, making it the most profitable season in the dataset.

2. Profit by Country

In [ ]:
Profit_by_Country = ( 
    bike_sales_df.groupby("Country")["Transaction_Profit"]
    .sum()
    .sort_values(ascending=False)
)

# Plot
plt.figure(figsize=(10, 5))
plt.bar(Profit_by_Country.index, Profit_by_Country.values, color='skyblue')
plt.xlabel("Country")
plt.ylabel("Total Profit ($)")
plt.title("Total Profit By Country")
plt.tight_layout()
plt.show()

The USA generates the highest total profit, making it the most profitable country in the dataset.

### Customer characteristics

1. Profit By Age Group

In [ ]:
Profit_by_Age_Group = ( 
    bike_sales_df.groupby("Age Group")["Transaction_Profit"]
    .sum()
    .sort_values(ascending=False)
)

# Plot
plt.figure(figsize=(10, 5))
plt.bar(Profit_by_Age_Group.index, Profit_by_Age_Group.values, color='skyblue')
plt.xlabel("Age Group")
plt.ylabel("Total Profit ($)")
plt.title("Total Profit By Age Group")
plt.tight_layout()
plt.show()

Adults (35–64) generate the highest total profit, making them the most profitable age group in the dataset.

2. Profit By Gender

In [ ]:
Profit_by_Gender = ( 
    bike_sales_df.groupby("Customer_Gender")["Transaction_Profit"]
    .sum()
    .sort_values(ascending=False)
)

# Plot
plt.figure(figsize=(8, 5))
plt.bar(Profit_by_Gender.index, Profit_by_Gender.values, color='skyblue')
plt.xlabel("Gender")
plt.ylabel("Total Profit ($)")
plt.title("Total Profit By Gender")
plt.tight_layout()
plt.show()

Male and female customers generate similar total profit, suggesting that gender is not a strong factor in overall profitability.

### Products

1. Profit by category

In [ ]:
Profit_by_Category = ( 
    bike_sales_df.groupby("Product_Category")["Transaction_Profit"]
    .sum()
    .sort_values(ascending=False)
)

# Plot
plt.figure(figsize=(8, 5))
plt.bar(Profit_by_Category.index, Profit_by_Category.values, color='skyblue')
plt.xlabel("Product Category")
plt.ylabel("Total Profit ($)")
plt.title("Total Profit By Category")
plt.tight_layout()
plt.show()

Bikes generate the highest total profit, making them the most profitable product category in the dataset.

2. Profit by individual product

In [ ]:
Profit_by_Product = ( 
    bike_sales_df.groupby("Product")["Transaction_Profit"]
    .sum()
    .sort_values(ascending=False)
    .head()
)

# Plot
plt.figure(figsize=(8, 5))
plt.bar(Profit_by_Product.index, Profit_by_Product.values, color='skyblue')
plt.xlabel("Product")
plt.ylabel("Total Profit ($)")
plt.title("Top 5 Most Profitable Products")
plt.tight_layout()
plt.xticks(rotation=45)
plt.show()

Mountain-200 appears four times among the top five most profitable products, indicating that it is a key driver of overall profitability.

# 6. Key Findings & Recommendations

### Key Findings
- Spring is the most profitable season, generating higher total profit than other periods of the year.
- The USA is the most profitable market, contributing the highest total profit among all countries.
- Adults (35–64) represent the most profitable customer age group.
- Profit contribution is relatively balanced between male and female customers, suggesting gender is not a key profitability driver.
- Bikes are the most profitable product category in the dataset.
- Mountain-200 stands out as a key profit driver, appearing multiple times among the most profitable products.

### Recommendations
- Increase marketing and promotional efforts during the spring season to capitalize on peak profitability.
- Focus sales strategies on the U.S. market while exploring growth opportunities in other high-performing countries like Australia.
- Target adult customers (35–64) with marketing campaigns, as they generate the highest profit.
- Prioritize inventory planning and product development around high-performing bike models such as Mountain-200.
